In [189]:
# loading package
import numpy as np
import pandas as pd
import igraph
from scipy.optimize import minimize
from scipy.special import digamma
import networkx as nx

## Loading data

Set the initial parameters for $\alpha, \beta, \delta, \gamma, \lambda$

In [14]:
np.random.seed(1)

p = 5
E_true  = np.zeros((p, p), dtype=int)

for j in range(1, p):
    E_true[j, j - 1] = 1

alpha_true = np.zeros((p, p))
alpha_true[E_true == 1] = np.random.uniform(0.5, 2, size=np.sum(E_true))
beta_true = np.zeros((p, p))
beta_true[E_true == 1] = np.random.uniform(-2, -0.5, size=np.sum(E_true))
delta_true = np.random.uniform(-1.5, -1, size=p)
gamma_true = np.random.uniform(1, 1.5, size=p)
lambda_true = np.exp(np.random.uniform(-2, 2, size=p))


# Generate synthetic data from the specified linear ZiG-DAG
n = 50  # Sample size
dat = np.zeros((n, p))
#order_nodes = list(igraph.topological_sort(igraph.Graph.Erdos_Renyi(n=p, p=0.5, directed=True)))
order_nodes = np.arange(0,5)

for j in order_nodes:
    pi_true = np.exp(np.dot(dat, alpha_true[j, :]) + delta_true[j])
    pi_true = pi_true / (1 + pi_true)
    pi_true[np.isnan(pi_true)] = 1
    mu_true = np.exp(np.dot(dat, beta_true[j, :]) + gamma_true[j])
    # use r output at first for following data
    #dat[:, j] = (1 - np.random.binomial(1, pi_true, size=n)) * rhP(n, lambda_true[j], mu_true)

dat = pd.read_csv('syn_data.csv',sep=',')
dat = np.array(dat)


**Linear ZiGDAG**

In [516]:
def f11_cpp(z, a, b, Iter, tol):
    
    fac = 1
    temp = fac
    series = 0
    
    for e in range(Iter):
        fac *= (a/b) * (z/(e+1))

        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (np.isfinite(series) or np.abs(series - temp) < tol): 
            return series
        
        temp = series
        
        a += 1
        b += 1
        
    return series

def f11_for_each(z, a, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = f11_cpp(z[i, 0], a, b, Iter, tol)
        
    return val

def log_ascfacto(z, n):
    p = len(n)
    out = np.zeros((p, 1))
    
    for i in range(p):
        for j in range(n[i]):
            out[i] += np.log(z+j)
            
    return out
    
    
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)


    #llmd = z[:,2*xp+2]

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])

    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    mod = np.mod(y[Y1].reshape(-1,1),np.log(Mu[Y1]).reshape(-1,1))
    mod[np.isnan(mod)] = 0

    
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + mod)
    
    if ((np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik

def gradF11b_cpp(z, b, Iter, tol):
    con = digamma(b) * f11_cpp(z, 1, b, Iter, tol)
    fac = digamma(b)
    temp = fac
    series = 0
    
    for i in range(Iter):
        fac *= (digamma(b+1)/digamma(b)) * (z/b)
        
        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (np.isfinite(series)) or (np.abs(series - temp) < tol):
            return (con - series)
        
        temp = series
        b += 1
        
    return (con-series)

def gradF11b_for_each(z, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = gradF11b_cpp(z[i, 0], b, Iter, tol)
        
    return val


def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)


    #llmd = z[:,2*xp+2]

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])

    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    X10 = x1[Y0];
    X11 = x1[Y1];
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    # check p1, p2, and p3
    
    grad = np.zeros(2 * pp + 3)
    # C++ code for p1
    #arma::sum(X10.each_col() % ((1 - Pi.elem(Y0)) % (Pi.elem(Y0) - Pi.elem(Y0) / eval1_F11.elem(Y0)) / dZIHP0), 0) - arma::sum(X11.each_col() % Pi.elem(Y1), 0);
    p1 = np.sum(X10 % ((1 - Pi[Y0]) % ((Pi[Y0] - (Pi[Y0]/ eval1_F11[Y0])) / dZIHP0)), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
    
    # C++ code for p2
    # -arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
    # arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    p2 = -np.sum(X10 % Mu[Y0] % ((1-Pi[Y0])/Lambda) % (eval2_F11[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum(X11 % (((Mu[Y1]/Lambda) % (eval2_F11[Y1]/eval1_F11[Y1])) -y[Y1].reshape(-1,1)), axis = 0)

    # C++ code for p3
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    p3 = (-np.sum((1-Pi[Y0]) % (eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    
    grad[:pp] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad

def create_dag(M):
    G = nx.DiGraph()
    snode = np.where(E_cand == 1)[0]
    enode = np.where(E_cand == 1)[1]
    
    for i in range(len(snode)):
        G.add_edge(snode[i], enode[i])
    
    return G

In [496]:
x = dat[: ,pa_j]
y = dat[:, j]
z = pars
z = z.reshape(1,-1)

pn, pp = x.shape
x1 = np.concatenate([x, np.ones((n,1))], axis = 1)


#llmd = z[:,2*xp+2]

Pi = np.exp(x1 @ z[:,0:pp+1].T)
Pi = Pi/(1+Pi)
Pi[~np.isfinite(Pi)] = 1

Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
Lambda = np.exp(z[:,-1])

Y0 = np.where(y<=1e-8)
Y1 = np.where(y>1e-8)

mod = np.mod(y[Y1].reshape(-1,1),np.log(Mu[Y1]).reshape(-1,1))
mod[np.isnan(mod)] = 0

llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)

In [521]:
minimize(fun= lambda z : dZIHP_cpp(z, dat[:, 0], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

      fun: -1.7976931348623157e+308
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([  0.        , 108.        , -34.63412698])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 1
      nit: 0
     njev: 1
   status: 2
  success: False
        x: array([0., 0., 0.])

In [556]:
bic_curr 


array([1., 1., 1., 1., 1.])

In [667]:
n, p = dat.shape
starting_dag = np.zeros((p, p))

bic_curr = np.empty(p)
est_curr = {
    "E": starting_dag,
    "alpha": np.zeros((p, p)),
    "beta": np.zeros((p, p)),
    "delta": np.zeros(p),
    "gamma": np.zeros(p),
    "lambda": np.ones(p),
}

for j in range(1):

    pa_j = (est_curr['E'][j, :] == 1)
    
    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    else:
        start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
        
    pars = np.array(start_j)
    
    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, 0], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
    
    p_j = sum(pa_j)
    bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

    if p_j > 0:
        
        est_curr["alpha"][j, pa_j] = out_j.x[0 : p_j]
        est_curr["beta"][j, pa_j]  = out_j.x[(p_j + 1) : (2 * p_j + 1)]

    est_curr["delta"][j]  = out_j.x[p_j]
    est_curr["gamma"][j]  = out_j.x[2 * p_j + 1]
    est_curr["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
    
    bic_iter = bic_curr
    est_iter = est_curr
    
    for Iter in range(1):
        IMPROV = False
        for j in range(p):
            for k in range(p):
                if j == k:
                    continue
                
                if est_curr["E"][j, k] == 0:
                    E_cand = est_curr["E"].copy()
                    E_cand[j, k] = 1
                    
                    # create dag for E_cand
                    G_cand = create_dag(E_cand)
                    if not nx.is_directed_acyclic_graph(G_cand): 
                        continue
                    
                    pa_j = (E_cand[j, ] == 1)
                    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    else:
                        start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    pars = np.array(start_j)
    
                    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, 0], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
    
                break
            break

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/123031670.py:31: RuntimeWarning: overflow encountered in double_scalars
  bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/123031670.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pars = np.array(start_j)


ValueError: could not broadcast input array from shape (2,) into shape (1,)

In [666]:
start_j

[array([0.]), 0.0, array([0.]), 0.0, 0.0]

In [652]:
G_cand = create_dag(E_cand)
nx.is_directed_acyclic_graph(G_cand)

True

In [645]:
E_cand[4, 1] = 1
E_cand[1, 0] = 1
E_cand

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [646]:
nx.is_directed_acyclic_graph(G)

True

In [535]:
for iter in range(1, 1 + 1):
    IMPROV = False

    # Search all DAGs reachable from the current DAG
    # Addition or deletion of an edge
    for j in range(p):
        for k in range(p):
            if j == k:
                continue

            if est_curr["E"][j, k] == 0:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                G_cand = nx.from_numpy_array(E_cand)

                if nx.is_directed_acyclic_graph(G_cand):
                    continue

                pa_j = E_cand[j, :] == 1
                start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]
                #start_j = np.concatenate([alpha, delta, beta, gamma, llmd], axis = 1)
                start_j = np.hstack([np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])])
                #print(start_j.shape)
                
                pars = np.array(start_j)
                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j]), x0=pars, method='BFGS')

                p_j = sum(pa_j)

                #bic_cand = bic_curr
                bic_cand = -2 * out_j.fun + (2*p_j + 3 )* np.log(n)

                update_para = out_j.x
                update_para = update_para.reshape(-1,1)

                tol = 1e-6

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    bic_iter = bic_cand
                    est_iter = est_curr
                    est_curr["E"] = E_cand
                    est_curr["alpha"][j, pa_j] = update_para[j, :p_j]
                    est_curr["beta"][j, pa_j] = update_para[j, p_j + 2:2 * p_j + 2]
                    est_curr["delta"][j] = update_para[j,:,p_j + 1][0]
                    est_curr["gamma"][j] = update_para[j,:,2 * p_j + 3][0]
                    est_curr["lambda"][j] = np.exp(update_para[:,p-1][0])
                
        print('----')
"""
tt = [1,2,3,4,5,6,7]
i = 2
tt[0:i]
tt[i+1:2*i+1]
tt[i]
tt[2*i+1]
tt[2*i+2]
"""

7

In [93]:
def hc_linear_zihp(dat, starting_dag, maxiter=500, tol=1e-12, optim_control=None, verbose=False):
    n, p = dat.shape

    if optim_control is None:
        optim_control = {"fnscale": -1, "maxiter": 10000, "reltol": 1.0e-8}

    if starting_dag is None:
        starting_dag = np.zeros((p, p))
        
    bic_curr = np.empty(p)
    est_curr = {
        "E": starting_dag,
        "alpha": np.zeros((p, p)),
        "beta": np.zeros((p, p)),
        "delta": np.zeros(p),
        "gamma": np.zeros(p),
        "lambda": np.ones(p),
    }
    
    return est_curr
    
    for j in range(p):
        
        pa_j = (est_curr['E'][j, :] == 1)

        start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
        
        
        if start_j[0][0:j].size == 0:
                alpha = np.zeros((p,j+1))
        else:
            alpha = start_j[0][0:p]

        if start_j[2][0:j].size == 0:
            beta = np.zeros((p,j+1))
        else:
            beta = start_j[2][0:p]

        if start_j[1] == 0:
            delta = np.zeros((p,1))
        else:
            delta = start_j[1]

        if start_j[3] == 0:
            gamma = np.zeros((p,1))
        else:
            gamma = start_j[3]

        if start_j[4] == 0:
            llmd = np.zeros((p,1))
        else:
            llmd = start_j[4]

        start_j = np.concatenate([alpha, delta, beta, gamma, llmd], axis = 1)

        pars = np.array(start_j)
        out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j]), x0=pars, method='BFGS')

        p_j = sum(pa_j)
        bic_curr = -2 * out_j.fun + (2*p_j + 3 )* np.log(n)
        
        update_para = out_j.x
        update_para = update_para.reshape((p, int(len(update_para)/p)))

        if p_j > 0:
            est_curr["alpha"][j, pa_j] = update_para[:p_j]
            est_curr["beta"][j, pa_j] = update_para[p_j + 2:2 * p_j + 2]
            
        est_curr["delta"][j] = update_para[:,p_j + 1][0]
        est_curr["gamma"][j] = update_para[:,2 * p_j + 3][0]
        est_curr["lambda"][j] = np.exp(update_para[:,p-1][0])

    bic_iter = bic_curr.copy()
    est_iter = est_curr.copy()

    
    return est_curr


In [73]:
hc_linear_zihp(dat, None)

{'E': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'alpha': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'beta': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'delta': array([0., 0., 0., 0., 0.]),
 'gamma': array([0., 0., 0., 0., 0.]),
 'lambda': array([1., 1., 1., 1., 1.])}

In [172]:
for iter in range(1, 1 + 1):
    IMPROV = False

    # Search all DAGs reachable from the current DAG
    # Addition or deletion of an edge
    for j in range(p):
        for k in range(p):
            if j == k:
                continue

            if est_curr["E"][j, k] == 0:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                G_cand = nx.from_numpy_array(E_cand)

                if nx.is_directed_acyclic_graph(G_cand):
                    continue

                pa_j = E_cand[j, :] == 1
                start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]
                #start_j = np.concatenate([alpha, delta, beta, gamma, llmd], axis = 1)
                start_j = np.hstack([np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])])
                #print(start_j.shape)
                
                pars = np.array(start_j)
                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j]), x0=pars, method='BFGS')

                p_j = sum(pa_j)

                #bic_cand = bic_curr
                bic_cand = -2 * out_j.fun + (2*p_j + 3 )* np.log(n)

                update_para = out_j.x
                update_para = update_para.reshape(-1,1)

                tol = 1e-6

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    bic_iter = bic_cand
                    est_iter = est_curr
                    est_curr["E"] = E_cand
                    est_curr["alpha"][j, pa_j] = update_para[j, :p_j]
                    est_curr["beta"][j, pa_j] = update_para[j, p_j + 2:2 * p_j + 2]
                    est_curr["delta"][j] = update_para[j,:,p_j + 1][0]
                    est_curr["gamma"][j] = update_para[j,:,2 * p_j + 3][0]
                    est_curr["lambda"][j] = np.exp(update_para[:,p-1][0])
                
        print('----')

4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:14: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(np.sum(x1 @ alpha.T, axis = 1))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:15: RuntimeWarning: invalid value encountered in true_divide
  Pi = Pi/(1+Pi)
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:23: RuntimeWarning: divide by zero encountered in log
  mod = np.mod(y[Y1],np.log(Mu[Y1]))


(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:14: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(np.sum(x1 @ alpha.T, axis = 1))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77

1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5)

...

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:14: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(np.sum(x1 @ alpha.T, axis = 1))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77

4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:14: RuntimeWarning: overflow encountered in exp
  Pi = np.exp(np.sum(x1 @ alpha.T, axis = 1))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77


(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1
4
(1, 5) 1


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_77153/1903604037.py:26: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
